In [1]:
import os
import sys

# Define local pass #
wd_path = '/home/hbkoziel/pyfesom/'

#sys.path.append("codes/modules") # add custom Vibe 's modules
sys.path.append('../..') # add standard 's modules
sys.path.append('../modules')
#########################################

In [2]:
import pyfesom as pf
import matplotlib.pyplot as plt
from matplotlib import cm
#sys.path.append('/home/hbkoziel/pyfesom/pyfesom/cmocean/')
#import cmocean as cmap
#from mpl_toolkits.basemap import Basemap
import numpy as np
from netCDF4 import Dataset
import time
#import seaborn as sns

import xarray as xr
import pandas as pd

No joblib
no cmocean


In [3]:
# Loading mesh for run

mesh_id    = 'meshArc4.5'
meshpath   = '/home/hbkoziel/pyfesom/data/mesh/'+mesh_id+'/'            # Defining path where mesh is stored
mesh = pf.load_mesh(meshpath, usepickle=True, get3d=True)                                    # Loading mesh, stores it in mesh.****  
#mesh = pf.fesom_mesh(meshpath, get3d=True)
#mesh.zlevs = -mesh.zlevs                                            # Depth is made negative

first_year = 1985
last_year  = 2015
years      = np.arange(first_year,last_year+1,1)
runid	= 'Arc12'

data_path  = '/scratch/usr/hbkoziel/'+runid+'/netcdf/'

/home/hbkoziel/pyfesom/data/mesh/meshArc4.5/pickle_mesh
2
The usepickle == True)
The pickle file for python 2 exists.
The mesh will be loaded from /home/hbkoziel/pyfesom/data/mesh/meshArc4.5/pickle_mesh


In [4]:
print('loading index...')
df = pd.read_csv('/home/hbkoziel/awi-models/data/mesh/meshArc4.5/bioregion_index_latlon.csv', delimiter=',',header=0)
print('LOADING DONE')
df.sort_values(by ='Unique_ID',inplace=True)
index = df["Unique_ID"]
index_unique = df.Unique_ID.unique().tolist()
index_unique.pop(-1)
print('SORTING...')
names = df.Name.unique().tolist()
names.pop(-1)
names[3]='Beaufort Sea - shelf'
names[4]='Amundsen-Viscount'
names.append('All Arctic Ocean (>66N)')
print('CLEANING DONE...')

loading index...
LOADING DONE
SORTING...
CLEANING DONE...


In [5]:
ncfile      = meshpath+'Arc4.5.initial.mesh.diag.nc'
f           = Dataset(ncfile, 'r')
NodalAreaModel = f.variables['cluster_area'][:]

In [6]:
ind_bio = df.full_index[df.Unique_ID == 24]

tracerlist = ['tr01','tr10','mixlay','cur50m','VNF','eke','NPP','icecon','PARmld','PAR3D','NNA','DINmld']

df2 = pd.DataFrame()

for ind in range(0,len(tracerlist)):
    print tracerlist[ind]
    dates = pd.date_range('1985', freq="MS", periods= 12*len(np.arange(1985,2016,1)))
    data = xr.open_mfdataset(data_path+tracerlist[ind]+'*.nc', concat_dim='time')
    data['time'] = dates
    data_sel = data[tracerlist[ind]].sel(time=slice(str(first_year)+'-01-01', str(last_year)+'-12-31'))
    data_sel = data_sel.compute()
    data_reg = np.sum(data_sel[:,ind_bio]*NodalAreaModel[ind_bio],axis=1)/np.sum(NodalAreaModel[ind_bio])
    df2[tracerlist[ind]] = data_reg.data
    if tracerlist[ind] == 'VNF':
        data_sel = data['w'].sel(time=slice(str(first_year)+'-01-01', str(last_year)+'-12-31'))
        data_sel = data_sel.compute()
        data_reg = np.sum(data_sel[:,ind_bio]*NodalAreaModel[ind_bio],axis=1)/np.sum(NodalAreaModel[ind_bio])
        df2['w'] = data_reg.data
    
        data_sel = data['DIN'].sel(time=slice(str(first_year)+'-01-01', str(last_year)+'-12-31'))
        data_sel = data_sel.compute()
        data_reg = np.sum(data_sel[:,ind_bio]*NodalAreaModel[ind_bio],axis=1)/np.sum(NodalAreaModel[ind_bio])
        df2['DIN'] = data_reg.data

df2['time'] = dates
df2.set_index('time', inplace= True)

tr01
tr10
mixlay
cur50m
VNF
eke
NPP
icecon
PARmld
PAR3D
NNA
DINmld


In [7]:
# Conversions
# df2['curspeed'] = np.sqrt(df2.u**2+df2.v**2)*100 # [cm s$^{-1}$]
# del df2['u']
# del df2['v']
df2['cur50m'] *= 100
df2['tr10'] = 12 * df2['tr10'] / 1000

In [8]:
# df2.head()

In [9]:
# Rename variables and units
df3 = df2.rename({'tr01': 'DINsurf',\
                  'tr10': 'ZB',\
                  'eke': 'EKE',\
                  'mixlay': 'MLD',\
                  'VNF': 'VNF',\
                  'icecon': 'SIC',\
                  'PARmld': 'PARmld',\
                  'PAR3D': 'PARsurf',\
                  'NNA': 'NNA',\
                  'NPP': 'NPP',\
                  'w':'wnf',\
                  'DIN': 'DINnf'},\
           axis=1)
df3.head()

,DINsurf,ZB,MLD,cur50m,VNF,wnf,DINnf,EKE,NPP,SIC,PARmld,PARsurf,NNA,DINmld
time,,,,,,,,,,,,,,
1985-01-01,5.025505,3.278505e-11,64.187938,2.920190,-0.186517,-0.115457,5.251850,5.846319,-0.000016,0.994961,2.230000e-16,2.230000e-16,0.000006,5.039335
1985-02-01,5.092333,6.690048e-13,67.928571,2.833709,3.551199,0.500548,5.367981,4.867825,-0.000011,0.996349,2.230000e-16,2.230000e-16,0.000004,5.110124
1985-03-01,5.168512,1.051836e-14,70.067242,3.204874,-0.103845,0.074891,5.450641,4.088163,-0.000008,0.995718,9.207157e-03,2.605001e-02,0.000004,5.184618
1985-04-01,5.202323,1.559222e-16,71.895586,3.167678,1.057146,0.086154,5.528432,2.664871,0.000004,0.997737,1.374558e-01,4.279646e-01,0.000002,5.222828
1985-05-01,5.194925,4.732647e-18,68.795493,3.216653,1.635694,0.232833,5.513505,1.602739,0.000025,0.995826,2.004728e-01,5.604649e-01,0.000002,5.221119


In [10]:
df3.to_pickle('Eurasian_basin_monthly')

In [11]:
tracerlist = {'ANPP','WAPE','SAPE'}
df4 = pd.DataFrame()

for tracername in tracerlist:
    print tracername
    dates = pd.date_range('1985', freq="YS", periods= len(np.arange(1985,2016,1)))
    
    if tracername == 'ANPP':
        data = xr.open_mfdataset(data_path+tracername+'*.nc', concat_dim='time')
        data['time'] = dates
        data_sel = data[tracername].sel(time=slice(str(first_year)+'-01-01', str(last_year)+'-12-31'))
        data_sel = data_sel.compute()
        data_reg = np.sum(data_sel[:,ind_bio]*NodalAreaModel[ind_bio],axis=1)
        df4[tracername] = data_reg.data *12 / 10**15
    elif tracername == 'WAPE':
        data = xr.open_mfdataset(data_path+tracername+'*monthly_v2.nc', concat_dim='time')
        data['time'] = dates
        data_sel = data['APE'].sel(time=slice(str(first_year)+'-01-01', str(last_year)+'-12-31'))
        data_sel = data_sel.compute()
        data_reg = np.sum(data_sel[:,ind_bio]*NodalAreaModel[ind_bio],axis=1)/np.sum(NodalAreaModel[ind_bio])
        df4[tracername] = data_reg.data
        data_sel = data['HLD'].sel(time=slice(str(first_year)+'-01-01', str(last_year)+'-12-31'))
        data_sel = data_sel.compute()
        data_reg = np.sum(data_sel[:,ind_bio]*NodalAreaModel[ind_bio],axis=1)/np.sum(NodalAreaModel[ind_bio])
        df4['WHLD'] = data_reg.data
    elif tracername == 'SAPE':
        data = xr.open_mfdataset(data_path+tracername+'*monthly_v2.nc', concat_dim='time')
        data['time'] = dates
        data_sel = data['APE'].sel(time=slice(str(first_year)+'-01-01', str(last_year)+'-12-31'))
        data_sel = data_sel.compute()
        data_reg = np.sum(data_sel[:,ind_bio]*NodalAreaModel[ind_bio],axis=1)/np.sum(NodalAreaModel[ind_bio])
        df4[tracername] = data_reg.data
        data_sel = data['HLD'].sel(time=slice(str(first_year)+'-01-01', str(last_year)+'-12-31'))
        data_sel = data_sel.compute()
        data_reg = np.sum(data_sel[:,ind_bio]*NodalAreaModel[ind_bio],axis=1)/np.sum(NodalAreaModel[ind_bio])
        df4['SHLD'] = data_reg.data 

df4['time'] = dates
df4.set_index('time', inplace= True)

ANPP
WAPE
SAPE


In [12]:
df4.to_pickle('Eurasian_basin_yearly')

In [13]:
tracerlist = {'EVNF'}
df4 = pd.DataFrame()

for tracername in tracerlist:
    print tracername
    dates = pd.date_range('2000', freq="MS", periods= 12*len(np.arange(2000,2016,1)))
    
    data = xr.open_mfdataset(data_path+tracername+'*monthly.nc', concat_dim='time')
    data['time'] = dates
    data_sel = data[tracername].sel(time=slice(str(first_year)+'-01-01', str(last_year)+'-12-31'))
    data_sel = data_sel.compute()
    data_reg = np.sum(data_sel[:,ind_bio]*NodalAreaModel[ind_bio],axis=1)/np.sum(NodalAreaModel[ind_bio])
    df4[tracername] = data_reg.data
     
df4['time'] = dates
df4.set_index('time', inplace= True)

EVNF


In [14]:
df4.to_pickle('Eurasian_basin_EVNF')

In [15]:
ind_bio = df.full_index[df.Unique_ID == 25]

tracerlist = ['tr01','tr10','mixlay','cur50m','VNF','eke','NPP','icecon','PARmld','PAR3D','NNA','DINmld']

df2 = pd.DataFrame()

for ind in range(0,len(tracerlist)):
    print tracerlist[ind]
    dates = pd.date_range('1985', freq="MS", periods= 12*len(np.arange(1985,2016,1)))
    data = xr.open_mfdataset(data_path+tracerlist[ind]+'*.nc', concat_dim='time')
    data['time'] = dates
    data_sel = data[tracerlist[ind]].sel(time=slice(str(first_year)+'-01-01', str(last_year)+'-12-31'))
    data_sel = data_sel.compute()
    data_reg = np.sum(data_sel[:,ind_bio]*NodalAreaModel[ind_bio],axis=1)/np.sum(NodalAreaModel[ind_bio])
    df2[tracerlist[ind]] = data_reg.data
    if tracerlist[ind] == 'VNF':
        data_sel = data['w'].sel(time=slice(str(first_year)+'-01-01', str(last_year)+'-12-31'))
        data_sel = data_sel.compute()
        data_reg = np.sum(data_sel[:,ind_bio]*NodalAreaModel[ind_bio],axis=1)/np.sum(NodalAreaModel[ind_bio])
        df2['w'] = data_reg.data
    
        data_sel = data['DIN'].sel(time=slice(str(first_year)+'-01-01', str(last_year)+'-12-31'))
        data_sel = data_sel.compute()
        data_reg = np.sum(data_sel[:,ind_bio]*NodalAreaModel[ind_bio],axis=1)/np.sum(NodalAreaModel[ind_bio])
        df2['DIN'] = data_reg.data

df2['time'] = dates
df2.set_index('time', inplace= True)

tr01
tr10
mixlay
cur50m
VNF
eke
NPP
icecon
PARmld
PAR3D
NNA
DINmld


In [16]:
# Conversions
# df2['curspeed'] = np.sqrt(df2.u**2+df2.v**2)*100 # [cm s$^{-1}$]
# del df2['u']
# del df2['v']
df2['cur50m'] *= 100
df2['tr10'] = 12 * df2['tr10'] / 1000

In [17]:
df2.head()

,tr01,tr10,mixlay,cur50m,VNF,w,DIN,eke,NPP,icecon,PARmld,PAR3D,NNA,DINmld
time,,,,,,,,,,,,,,
1985-01-01,4.067230,6.297426e-11,47.111587,2.526962,-0.426529,-0.081928,4.525225,4.946413,-1.252058e-05,0.995733,0.000561,0.000740,0.000005,4.095412
1985-02-01,4.172786,1.723739e-12,50.831493,2.179327,2.618317,0.590252,4.849525,2.107308,-6.833990e-06,0.996165,0.007387,0.013761,0.000003,4.224850
1985-03-01,4.294595,1.493632e-14,53.859722,2.360097,0.730947,0.186309,5.131526,2.440675,-4.308777e-07,0.996816,0.220934,0.513057,0.000002,4.359552
1985-04-01,4.365262,2.868283e-16,55.359655,2.552073,0.954442,0.152958,5.322186,1.495896,1.602278e-05,0.995300,0.433908,0.973652,0.000002,4.449477
1985-05-01,4.379881,1.316507e-17,49.492041,2.396575,1.196109,0.246432,5.015919,1.626018,1.228533e-04,0.981938,2.957933,5.670387,0.000007,4.448718


In [18]:
# Rename variables and units
df3 = df2.rename({'tr01': 'DINsurf',\
                  'tr10': 'ZB',\
                  'eke': 'EKE',\
                  'mixlay': 'MLD',\
                  'VNF': 'VNF',\
                  'icecon': 'SIC',\
                  'PARmld': 'PARmld',\
                  'PAR3D': 'PARsurf',\
                  'NNA': 'NNA',\
                  'NPP': 'NPP',\
                  'w':'wnf',\
                  'DIN': 'DINnf'},\
           axis=1)
df3.head()

,DINsurf,ZB,MLD,cur50m,VNF,wnf,DINnf,EKE,NPP,SIC,PARmld,PARsurf,NNA,DINmld
time,,,,,,,,,,,,,,
1985-01-01,4.067230,6.297426e-11,47.111587,2.526962,-0.426529,-0.081928,4.525225,4.946413,-1.252058e-05,0.995733,0.000561,0.000740,0.000005,4.095412
1985-02-01,4.172786,1.723739e-12,50.831493,2.179327,2.618317,0.590252,4.849525,2.107308,-6.833990e-06,0.996165,0.007387,0.013761,0.000003,4.224850
1985-03-01,4.294595,1.493632e-14,53.859722,2.360097,0.730947,0.186309,5.131526,2.440675,-4.308777e-07,0.996816,0.220934,0.513057,0.000002,4.359552
1985-04-01,4.365262,2.868283e-16,55.359655,2.552073,0.954442,0.152958,5.322186,1.495896,1.602278e-05,0.995300,0.433908,0.973652,0.000002,4.449477
1985-05-01,4.379881,1.316507e-17,49.492041,2.396575,1.196109,0.246432,5.015919,1.626018,1.228533e-04,0.981938,2.957933,5.670387,0.000007,4.448718


In [19]:
df3.to_pickle('Amerasian_basin_monthly')

In [20]:
tracerlist = {'ANPP','WAPE','SAPE'}
df4 = pd.DataFrame()

for tracername in tracerlist:
    print tracername
    dates = pd.date_range('1985', freq="YS", periods= len(np.arange(1985,2016,1)))
    
    if tracername == 'ANPP':
        data = xr.open_mfdataset(data_path+tracername+'*.nc', concat_dim='time')
        data['time'] = dates
        data_sel = data[tracername].sel(time=slice(str(first_year)+'-01-01', str(last_year)+'-12-31'))
        data_sel = data_sel.compute()
        data_reg = np.sum(data_sel[:,ind_bio]*NodalAreaModel[ind_bio],axis=1)
        df4[tracername] = data_reg.data *12 / 10**15
    elif tracername == 'WAPE':
        data = xr.open_mfdataset(data_path+tracername+'*monthly_v2.nc', concat_dim='time')
        data['time'] = dates
        data_sel = data['APE'].sel(time=slice(str(first_year)+'-01-01', str(last_year)+'-12-31'))
        data_sel = data_sel.compute()
        data_reg = np.sum(data_sel[:,ind_bio]*NodalAreaModel[ind_bio],axis=1)/np.sum(NodalAreaModel[ind_bio])
        df4[tracername] = data_reg.data
        data_sel = data['HLD'].sel(time=slice(str(first_year)+'-01-01', str(last_year)+'-12-31'))
        data_sel = data_sel.compute()
        data_reg = np.sum(data_sel[:,ind_bio]*NodalAreaModel[ind_bio],axis=1)/np.sum(NodalAreaModel[ind_bio])
        df4['WHLD'] = data_reg.data
    elif tracername == 'SAPE':
        data = xr.open_mfdataset(data_path+tracername+'*monthly_v2.nc', concat_dim='time')
        data['time'] = dates
        data_sel = data['APE'].sel(time=slice(str(first_year)+'-01-01', str(last_year)+'-12-31'))
        data_sel = data_sel.compute()
        data_reg = np.sum(data_sel[:,ind_bio]*NodalAreaModel[ind_bio],axis=1)/np.sum(NodalAreaModel[ind_bio])
        df4[tracername] = data_reg.data
        data_sel = data['HLD'].sel(time=slice(str(first_year)+'-01-01', str(last_year)+'-12-31'))
        data_sel = data_sel.compute()
        data_reg = np.sum(data_sel[:,ind_bio]*NodalAreaModel[ind_bio],axis=1)/np.sum(NodalAreaModel[ind_bio])
        df4['SHLD'] = data_reg.data 

df4['time'] = dates
df4.set_index('time', inplace= True)

ANPP
WAPE
SAPE


In [21]:
df4.to_pickle('Amerasian_basin_yearly')

In [22]:
# UNITS

# DIN [mmol m$^{-3}$]
# Zooplankton biomass [mg C m$^{-2}$]
# Eddy kinetic energey [cm$^{2}$ s$^{-2}$]
# Mixed Layer Depth [m]
# Vertical nitrate fluxes [mmol m$^{-2}$ month$^{-1}$]
# Sea ice concentration [%]
# PAR$_{MLD}$ [W m$^{-2}$]
# PAR$_{surf}$ [W m$^{-2}$]
# Phytoplankton N-Assimilation [mmol m$^{-2}$ d$^{-1}$]
# Net Primary Production [mmol m$^{-2}$ d$^{-1}$]
# DIN$_{MLD}$ [mmol m$^{-3}$]

In [23]:
tracerlist = {'EVNF'}
df4 = pd.DataFrame()

for tracername in tracerlist:
    print tracername
    dates = pd.date_range('2000', freq="MS", periods= 12*len(np.arange(2000,2016,1)))
    
    data = xr.open_mfdataset(data_path+tracername+'*monthly.nc', concat_dim='time')
    data['time'] = dates
    data_sel = data[tracername].sel(time=slice(str(first_year)+'-01-01', str(last_year)+'-12-31'))
    data_sel = data_sel.compute()
    data_reg = np.sum(data_sel[:,ind_bio]*NodalAreaModel[ind_bio],axis=1)/np.sum(NodalAreaModel[ind_bio])
    df4[tracername] = data_reg.data
     
df4['time'] = dates
df4.set_index('time', inplace= True)

EVNF


In [24]:
df4.to_pickle('Amerasian_basin_EVNF')

In [25]:
# ind_bio = df.full_index[df.Unique_ID == 24]

# tracerlist = ['VNF']

# df2 = pd.DataFrame()

# for ind in range(0,len(tracerlist)):
#     print tracerlist[ind]
#     dates = pd.date_range('1985', freq="MS", periods= 12*len(np.arange(1985,2016,1)))
#     data1 = xr.open_mfdataset(data_path+tracerlist[ind]+'*.nc', concat_dim='time')
#     #data2 = xr.open_mfdataset(data_path+'cur50m'+'*.nc', concat_dim='time')
#     data1['time'] = dates
#     #data2['time'] = dates
#     #data_masked = data1.where((data2['cur50m'] >= .001) & (data1['w'] != 0)) 
#     #data[data_curr < 0.03] = np.nan
#     data_sel = data1['VNF'].sel(time=slice(str(first_year)+'-01-01', str(last_year)+'-12-31'))
#     data_sel = data_sel.compute()
#     data_reg = np.sum(data_sel[:,ind_bio]*NodalAreaModel[ind_bio],axis=1)/np.sum(NodalAreaModel[ind_bio])
#     df2['VNF'] = data_reg.data
    
#     data_sel = data1['w'].sel(time=slice(str(first_year)+'-01-01', str(last_year)+'-12-31'))
#     data_sel = data_sel.compute()
#     data_reg = np.sum(data_sel[:,ind_bio]*NodalAreaModel[ind_bio],axis=1)/np.sum(NodalAreaModel[ind_bio])
#     df2['w'] = data_reg.data
    
#     data_sel = data1['DIN'].sel(time=slice(str(first_year)+'-01-01', str(last_year)+'-12-31'))
#     data_sel = data_sel.compute()
#     data_reg = np.sum(data_sel[:,ind_bio]*NodalAreaModel[ind_bio],axis=1)/np.sum(NodalAreaModel[ind_bio])
#     df2['DIN'] = data_reg.data

# df2['time'] = dates
# df2.set_index('time', inplace= True)

In [26]:
# cur50m_mean = data2['cur50m'].mean(dim='time')
# cur50m_mean = cur50m_mean.compute()
# plt.plot(mesh.x2[ind_bio],mesh.y2[ind_bio],'.')

In [27]:
# df2.DIN.plot()

In [28]:
# df2.VNF.plot()

In [29]:
# df2.w.plot()
# #df2.VNF.plot()

In [30]:
# plt.plot(df2.w*df2.DIN)
# plt.plot(df2.VNF)

In [31]:
# df2_q = df2['w'].resample('Q').mean()
# df2_qstd = df2['w'].resample('Q').std()

# df3_q = df2['VNF'].resample('Q').mean()
# df3_qstd = df2['VNF'].resample('Q').std()
# # plt.plot(df2[df2.index.month == 1].w,'k')
# # plt.plot(df2[df2.index.month == 2].w,'k')
# # plt.plot(df2[df2.index.month == 10].w,'k')

# # plt.plot(df2_q[df2_q.index.month == 3],lw =5)
# #plt.plot(df3_qstd[df3_qstd.index.month == 3])

# #plt.plot(df2_qstd[df2_qstd.index.month == 6])
# #plt.plot(df2_qstd[df2_qstd.index.month == 9])
# #plt.plot(df2_qstd[df2_qstd.index.month == 12])

# min_ser = df2_q -0.5 *df2_qstd
# max_ser = df2_q +0.5 *df2_qstd

# plt.plot(df2_q, lw =5)
# plt.plot(min_ser,'k--')
# plt.plot(max_ser,'k--')

# plt.plot(df2_qstd[df2_qstd.index.month == 3])

In [32]:
# plt.plot(df3_q[df3_q.index.month == 3])
# plt.plot(df2_q[df2_q.index.month == 3])

In [33]:
# plt.plot(df2['w'].resample('Y').mean())

# min_ser = df2['w'].resample('Y').mean() -0.5 *df2['w'].resample('Y').std()
# max_ser = df2['w'].resample('Y').mean() +0.5 *df2['w'].resample('Y').std()

# plt.plot(min_ser,'k--')
# plt.plot(max_ser,'k--')

# plt.plot(df2['w'].resample('Y').std())